In [1]:
# ---------------------
# EDIT THESE BEFORE RUNNING
PROJECT_ID = "unext-1660103646868"         # <--- e.g. "my-vertex-demo"
SERVICE_ACCOUNT_JSON = r"D:\SUB\NubeEra_work\GCP-NuvePro\unext-1660103646868-4283df77df53.json"  # <--- upload your service account JSON to Colab, then set path
LOCATION = "us-central1"               # recommended region (change if needed)
# ---------------------


## 1. Install required libraries


In [2]:
!pip install --upgrade google-cloud-aiplatform google-cloud-documentai google-cloud-storage google-cloud-retail faiss-cpu


  Using cached google_cloud_storage-3.4.1-py3-none-any.whl.metadata (13 kB)


    scikit-learn (>=1.0.*) ; extra == 'pipelines'
                  ~~~~~~^


## 2. Authenticate with service account


In [3]:
import os
assert PROJECT_ID != "YOUR_PROJECT_ID", "Please set PROJECT_ID at the top of the notebook"
assert os.path.exists(SERVICE_ACCOUNT_JSON), f"Service account JSON not found at {SERVICE_ACCOUNT_JSON}"

# Set environment variables for gcloud style auth and client libs
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = SERVICE_ACCOUNT_JSON
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

print("GCP project set to:", PROJECT_ID)
print("Credentials path:", SERVICE_ACCOUNT_JSON)


GCP project set to: unext-1660103646868
Credentials path: D:\SUB\NubeEra_work\GCP-NuvePro\unext-1660103646868-4283df77df53.json


## 5. Model Garden demo — text generation with `text-bison`
**Goal**: Use pre-trained foundation model to generate text. We'll use `google-cloud-aiplatform` Python client.

> NOTE: Model names in Model Garden might update. If `text-bison@001` fails, check the Model Garden console or `aiplatform.list_models()` for available pretrained IDs.


In [4]:
from google.cloud import aiplatform
aiplatform.init(project=PROJECT_ID, location=LOCATION)

# Example with the TextGenerationModel wrapper
try:
    model = aiplatform.TextGenerationModel.from_pretrained("text-bison@001")
    prompt = "Explain how Retail personalization improves customer experience in 3 simple bullet points."
    resp = model.predict(prompt, max_output_tokens=256)
    print("=== Model Garden (text-bison) output ===")
    print(resp.text)
except Exception as e:
    print("Model Garden demo failed:", e)
    print("Possible causes: model name changed or client library updated. You can list available models with aiplatform models list or check Model Garden in Console.")


Model Garden demo failed: module 'google.cloud.aiplatform' has no attribute 'TextGenerationModel'
Possible causes: model name changed or client library updated. You can list available models with aiplatform models list or check Model Garden in Console.


In [6]:
import numpy, pandas, grpc, google.cloud.aiplatform as aiplatform
print("NumPy:", numpy.__version__)
print("Pandas:", pandas.__version__)
print("gRPC:", grpc.__version__)
print("Vertex AI SDK:", aiplatform.__version__)


NumPy: 1.26.4
Pandas: 2.2.2
gRPC: 1.64.1
Vertex AI SDK: 1.122.0


In [8]:
from google.cloud import aiplatform
aiplatform.init(project="unext-1660103646868", location="us-central1")

for model in aiplatform.Model.list():
    print(model.display_name)
    


In [14]:
!pip install -q faiss-cpu vertexai google-cloud-aiplatform


    scikit-learn (>=1.0.*) ; extra == 'pipelines'
                  ~~~~~~^


In [16]:
!pip uninstall -y vertexai

Found existing installation: vertexai 1.66.0
Uninstalling vertexai-1.66.0:
  Successfully uninstalled vertexai-1.66.0


In [17]:
!pip install --upgrade google-genai

    scikit-learn (>=1.0.*) ; extra == 'pipelines'
                  ~~~~~~^


2️⃣ Imports & Initialization

In [24]:
from google.genai import Client
from google.cloud import aiplatform

# Initialize Vertex AI for embeddings and client
PROJECT_ID = "unext-1660103646868"
LOCATION = "us-central1"
SERVICE_ACCOUNT_JSON = r"D:\SUB\NubeEra_work\GCP-NuvePro\unext-1660103646868-4283df77df53.json"  # your service account JSON

# Set environment variables for authentication
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = SERVICE_ACCOUNT_JSON

# Initialize Vertex AI SDK (needed for embeddings)
aiplatform.init(project=PROJECT_ID, location=LOCATION)

# Initialize Google Gen AI client using Vertex AI credentials
client = Client(vertexai=True, project=PROJECT_ID, location=LOCATION)


In [27]:
prompt = "Explain what Vertex AI Agent Builder is in simple terms."
response = client.models.generate_content(
    model="gemini-2.0-flash-001",
    contents=[prompt]
)
print(response.text)


Imagine you want to build a helpful robot that can answer questions, book appointments, or complete tasks for your customers. Vertex AI Agent Builder is like a set of tools and building blocks that Google provides to make creating that robot much easier.

Instead of having to write complex code from scratch, you can use Agent Builder to:

*   **Define what your robot needs to know:** You feed it information like documents, websites, or databases.
*   **Teach it how to understand questions:**  It uses advanced AI to understand the meaning behind user questions, even if they're phrased differently.
*   **Design how it should respond and take action:** You can customize the robot's responses and tell it how to perform specific tasks (like booking a flight).
*   **Test and improve it easily:** Agent Builder allows you to test and refine your robot's performance to make it more accurate and helpful over time.

**In short, Vertex AI Agent Builder is a platform that helps you build and deploy

3️⃣ Prepare Knowledge Base

In [28]:
documents = [
    "Vertex AI lets you train, tune, and deploy ML models at scale on Google Cloud.",
    "Vertex AI Agent Builder allows creating conversational AI agents using retrieval-augmented generation.",
    "Document AI can extract structured data from unstructured documents like invoices.",
    "Retail API helps personalize product recommendations and search for e-commerce applications."
]


4️⃣ Create Embeddings

In [30]:
import os
import numpy as np
import faiss
import textwrap

from google.cloud import aiplatform
from google.genai import Client
from vertexai.language_models import TextEmbeddingModel

# Load Vertex AI embedding model
embed_model = TextEmbeddingModel.from_pretrained("gemini-embedding-001")

# Generate embeddings
embeddings = embed_model.get_embeddings(documents)
vectors = np.array([e.values for e in embeddings]).astype("float32")
print("✅ Created embeddings shape:", vectors.shape)


✅ Created embeddings shape: (4, 3072)


5️⃣ Build FAISS Index

In [31]:
dimension = vectors.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(vectors)
print(f"✅ Added {index.ntotal} vectors to FAISS index")


✅ Added 4 vectors to FAISS index


6️⃣ Define Retrieval Function

In [32]:
def retrieve_similar_docs(query, k=2):
    q_emb = embed_model.get_embeddings([query])[0].values
    q_vec = np.array([q_emb]).astype("float32")
    distances, indices = index.search(q_vec, k)
    return [documents[i] for i in indices[0]]


7️⃣ Define RAG Query Function

In [35]:
def ask_with_rag(query):
    # Retrieve top-k relevant documents
    context_docs = retrieve_similar_docs(query)
    context = "\n\n".join(context_docs)
    
    # Prepare prompt
    prompt = f"Use the context below to answer the question.\n\nContext:\n{context}\n\nQuestion: {query}\nAnswer:"
    
    # Generate response using Gen AI
    response = client.models.generate_content(
        model="gemini-2.0-flash-001",
        contents=[prompt]
    )
    return response.text


8️⃣ Test Your RAG Agent

In [36]:
question = "How can I create a conversational AI agent with Vertex AI?"
answer = ask_with_rag(question)

print("Question:", question)
print("\nAnswer:")
print(textwrap.fill(answer, width=100))


Question: How can I create a conversational AI agent with Vertex AI?

Answer:
You can use Vertex AI Agent Builder to create conversational AI agents.
